In [2]:
# librerías 
import re
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

In [27]:
prime_df = pd.read_csv("./amazon_prime_titles.csv", sep=",")
prime_df["streaming_service"] = "Amazon Prime"
prime_df = prime_df.rename(columns={"listed_in": "genre"})
print(prime_df.info())
display(prime_df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   show_id            9668 non-null   object
 1   type               9668 non-null   object
 2   title              9668 non-null   object
 3   director           7586 non-null   object
 4   cast               8435 non-null   object
 5   country            672 non-null    object
 6   date_added         155 non-null    object
 7   release_year       9668 non-null   int64 
 8   rating             9331 non-null   object
 9   duration           9668 non-null   object
 10  genre              9668 non-null   object
 11  description        9668 non-null   object
 12  streaming_service  9668 non-null   object
dtypes: int64(1), object(12)
memory usage: 982.0+ KB
None


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,genre,description,streaming_service
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...,Amazon Prime
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...,Amazon Prime
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...,Amazon Prime
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ...",Amazon Prime
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...,Amazon Prime


In [28]:
# Convertimos las fechas a tipo fecha
prime_df["date_added"] = pd.to_datetime(prime_df["date_added"])

# Extraemos el Año, Mes, Día de la fecha que se añadio a la plataforma
prime_df["year_added"] = prime_df["date_added"].dt.strftime('%Y')
prime_df["month_added"] = prime_df["date_added"].dt.strftime('%m')
prime_df["day_added"] = prime_df["date_added"].dt.strftime('%d')

# Convertimos a lista los países y géneros
prime_df["country"] = prime_df["country"].str.split(", ")
prime_df["genre"] = prime_df["genre"].str.split(", ")

# Dividimos la columna duration en dos:
# ['duration'] contiene el núm. de minutos o temporadas.
# ['duration_type'] contiene el el tipo de duración. de min (minutos) o Seasons (temporadas).
prime_df[['duration','duration_type']] = prime_df["duration"].str.split(" ",expand=True,)

display(prime_df.head())

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,genre,description,streaming_service,year_added,month_added,day_added,duration_type
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",[Canada],2021-03-30,2014,NaN,113,"[Comedy, Drama]",A small fishing village must procure a local d...,Amazon Prime,2021,03,30,min
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",[India],2021-03-30,2018,13+,110,"[Drama, International]",A Metro Family decides to fight a Cyber Crimin...,Amazon Prime,2021,03,30,min
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",[United States],2021-03-30,2017,NaN,74,"[Action, Drama, Suspense]",After a man discovers his wife is cheating on ...,Amazon Prime,2021,03,30,min
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",[United States],2021-03-30,2014,NaN,69,[Documentary],"Pink breaks the mold once again, bringing her ...",Amazon Prime,2021,03,30,min
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",[United Kingdom],2021-03-30,1989,NaN,45,"[Drama, Fantasy]",Teenage Matt Banting wants to work with a famo...,Amazon Prime,2021,03,30,min


In [29]:
prime_df['rating'].unique()

array([nan, '13+', 'ALL', '18+', 'R', 'TV-Y', 'TV-Y7', 'NR', '16+',
       'TV-PG', '7+', 'TV-14', 'TV-NR', 'TV-G', 'PG-13', 'TV-MA', 'G',
       'PG', 'NC-17', 'UNRATED', '16', 'AGES_16_', 'AGES_18_', 'ALL_AGES',
       'NOT_RATE'], dtype=object)

Amazon prime (Rating: https://www.primevideo.com/help/ref=atv_hp_nd_cnt?nodeId=GFGQU3WYEG6FSJFJ )

In [30]:
# APTA = ALL_AGES, ALL, G
apta_list = ['ALL_AGES', 'ALL', 'G', 'TV-G', 'TV-Y',]

# 7 (7+) = 7+, PG
siete_list = ['7+', 'PG', 'TV-PG', 'TV-Y7', 'TV-Y7-FV']

# 12 (13+) = 13+
doce_list = ['13+', 'PG-13']
# 16 (16+) = 16+, AGES_16_, 16
mayores_16_list = ['16+', 'AGES_16_', '16', 'TV-14']
# 18 (18+) = 18+, AGES_18_, NC-17, R
mayores_18_list = ['18+', 'AGES_18_', 'NC-17', 'R', 'TV-MA']
# Sin calificación = NOT_RATE, UNRATED, NR, nan
sc_list = ['NOT_RATE', 'UNRATED', 'NR', 'nan', 'TV-NR']

# Condiciones para renombrar la clasificación por edades de las películas
conditions = [prime_df['rating'].isin(apta_list),
              prime_df['rating'].isin(siete_list),
              prime_df['rating'].isin(doce_list),
              prime_df['rating'].isin(mayores_16_list),
              prime_df['rating'].isin(mayores_18_list),
              prime_df['rating'].isin(sc_list),
             ]
# Resultado de la clasificación anterior por edades de las películas
results = ['APTA', 
           '7', 
           '12', 
           '16', 
           '18', 
           'Sin calificación'
          ]

# Renombramos la clasificación por edades
prime_df['rating'] = np.select(conditions,results, default='Sin calificación')

display(prime_df.head())

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,genre,description,streaming_service,year_added,month_added,day_added,duration_type
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",[Canada],2021-03-30,2014,Sin calificación,113,"[Comedy, Drama]",A small fishing village must procure a local d...,Amazon Prime,2021,03,30,min
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",[India],2021-03-30,2018,12,110,"[Drama, International]",A Metro Family decides to fight a Cyber Crimin...,Amazon Prime,2021,03,30,min
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",[United States],2021-03-30,2017,Sin calificación,74,"[Action, Drama, Suspense]",After a man discovers his wife is cheating on ...,Amazon Prime,2021,03,30,min
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",[United States],2021-03-30,2014,Sin calificación,69,[Documentary],"Pink breaks the mold once again, bringing her ...",Amazon Prime,2021,03,30,min
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",[United Kingdom],2021-03-30,1989,Sin calificación,45,"[Drama, Fantasy]",Teenage Matt Banting wants to work with a famo...,Amazon Prime,2021,03,30,min


In [31]:
prime_df['rating'].unique()

array(['Sin calificación', '12', 'APTA', '18', '7', '16'], dtype=object)

In [32]:
# Dividimos en dos dataframes
prime_movies_df = prime_df[prime_df['type'] == 'Movie'].copy()
print(f"Movies: {prime_movies_df.shape}")
prime_series_df = prime_df[prime_df['type'] == 'TV Show'].copy()
print(f"Series: {prime_series_df.shape}")

Movies: (7814, 17)
Series: (1854, 17)


## Juntamos con IMBD

In [33]:
imbd_df = pd.read_csv("./imbd_data.csv", sep=";")
print(imbd_df.info())
display(imbd_df.head(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1192579 entries, 0 to 1192578
Data columns (total 12 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   Unnamed: 0      1192579 non-null  int64  
 1   tconst          1192579 non-null  object 
 2   titleType       1192579 non-null  object 
 3   primaryTitle    1192579 non-null  object 
 4   originalTitle   1192579 non-null  object 
 5   isAdult         1192579 non-null  int64  
 6   startYear       1192579 non-null  object 
 7   endYear         1192579 non-null  object 
 8   runtimeMinutes  1192579 non-null  object 
 9   genres          1192577 non-null  object 
 10  averageRating   1192579 non-null  float64
 11  numVotes        1192579 non-null  int64  
dtypes: float64(1), int64(3), object(8)
memory usage: 109.2+ MB
None


,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",5.7,1844
1,1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",6.0,236
2,2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",6.5,1603


In [34]:
# Películas sin año de salida
imbd_df_no_year = imbd_df[imbd_df['startYear'] == '\\N']
# Películas con año de salida
imbd_df = imbd_df[imbd_df['startYear'] != '\\N']
imbd_df['startYear'] = imbd_df['startYear'].astype('int64')
display(imbd_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1192379 entries, 0 to 1192578
Data columns (total 12 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   Unnamed: 0      1192379 non-null  int64  
 1   tconst          1192379 non-null  object 
 2   titleType       1192379 non-null  object 
 3   primaryTitle    1192379 non-null  object 
 4   originalTitle   1192379 non-null  object 
 5   isAdult         1192379 non-null  int64  
 6   startYear       1192379 non-null  int64  
 7   endYear         1192379 non-null  object 
 8   runtimeMinutes  1192379 non-null  object 
 9   genres          1192379 non-null  object 
 10  averageRating   1192379 non-null  float64
 11  numVotes        1192379 non-null  int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 118.3+ MB


None

In [38]:
# Covertir los títulos de las películas a minuscula para evitar fallos al unir
prime_df['title'] = prime_df['title'].str.lower()
imbd_df['originalTitle'] = imbd_df['originalTitle'].str.lower()

In [39]:
# Juntamos con Prime Video con IMBD
merged_df = pd.merge(prime_df, 
                     imbd_df, 
                     left_on=['title', 'release_year'],
                     right_on=['originalTitle', 'startYear'],)
merged_df.shape

(4785, 29)

In [36]:
merged_df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,...,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",[India],2021-03-30,2018,12,110,...,movie,Take Care Good Night,Take Care Good Night,0,2018,\N,110,"Crime,Drama,Family",7.5,207
1,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",[United States],2021-03-30,2017,Sin calificación,74,...,movie,Secrets of Deception,Secrets of Deception,0,2017,\N,76,"Action,Crime,Drama",4.3,289
2,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",[United Kingdom],2021-03-30,1989,Sin calificación,45,...,tvEpisode,Monster Maker,Monster Maker,0,1989,\N,45,"Comedy,Family,Fantasy",7.1,110
3,s8,Movie,Grease Live!,"Thomas Kail, Alex Rudzinski","Julianne Hough, Aaron Tveit, Vanessa Hudgens, ...",[United States],2021-03-30,2016,Sin calificación,131,...,tvMovie,Grease Live!,Grease Live!,0,2016,\N,140,"Comedy,Musical,Romance",7.3,9195
4,s9,Movie,Global Meltdown,Daniel Gilboy,"Michael Paré, Leanne Khol Young, Patrick J. Ma...",[Canada],2021-03-30,2017,Sin calificación,87,...,tvMovie,Global Meltdown,Global Meltdown,0,2017,\N,90,"Action,Adventure,Drama",3.2,568
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4371,s9660,Movie,10 Things I Hate About You,Gil Junger,"Heath Ledger, Julia Stiles, Joseph Gordon-Levi...",NaN,NaT,1999,12,97,...,movie,10 Things I Hate About You,10 Things I Hate About You,0,1999,\N,97,"Comedy,Drama,Romance",7.3,318635
4372,s9662,TV Show,Terrahawks,NaN,"DENISE BRYER, WINDSOR DAVIES, JEREMY HITCHEN, ...",NaN,NaT,1983,APTA,3,...,tvSeries,Terrahawks,Terrahawks,0,1983,1986,23,"Family,Sci-Fi",7.2,716
4373,s9663,Movie,River,Emily Skye,"Mary Cameron Rogers, Alexandra Rose, Rob Marsh...",NaN,NaT,2021,16,93,...,movie,River,River,0,2021,\N,93,"Drama,Sci-Fi",3.1,170
4374,s9663,Movie,River,Emily Skye,"Mary Cameron Rogers, Alexandra Rose, Rob Marsh...",NaN,NaT,2021,16,93,...,movie,River,River,0,2021,\N,75,Documentary,8.3,17
